# Description

It projects input data into a UMAP representation.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from IPython.display import display

import pandas as pd
import umap
import matplotlib.pyplot as plt
import seaborn as sns

import conf
from utils import generate_result_set_name

# Settings

In [3]:
INPUT_FILEPATH_STEM = 'smultixcan-efo_partial-mashr-zscores'

In [4]:
INPUT_FILEPATH = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    'z_score_std',
    f'z_score_std-{INPUT_FILEPATH_STEM}.pkl',
).resolve()
display(INPUT_FILEPATH)

input_filepath_stem = INPUT_FILEPATH.stem
display(input_filepath_stem)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/data_transformations/z_score_std/z_score_std-smultixcan-efo_partial-mashr-zscores.pkl')

'z_score_std-smultixcan-efo_partial-mashr-zscores'

In [5]:
# number of components to use in the dimensionality reduction step
DR_OPTIONS = {
    'n_components': [5, 10, 20, 30, 40, 50],
    'metric': 'euclidean',
    'n_neighbors': 15,
    'random_state': 0,
}

In [6]:
# output dir for this notebook
RESULTS_DIR = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    'umap'
).resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/data_transformations/umap')

In [7]:
# dictionary containing all options/settings (used to generate filenames)
ALL_OPTIONS = DR_OPTIONS.copy()
# ALL_OPTIONS['proj_percentile'] = PERCENTILE_NAME

display(ALL_OPTIONS)

{'n_components': [5, 10, 20, 30, 40, 50],
 'metric': 'euclidean',
 'n_neighbors': 15,
 'random_state': 0}

# Load input file

In [8]:
data = pd.read_pickle(INPUT_FILEPATH)

In [9]:
data.shape

(3749, 6452)

In [10]:
data.head()

gene_name,ENSG00000183087,ENSG00000157227,ENSG00000096696,ENSG00000175130,ENSG00000113140,ENSG00000117984,ENSG00000116016,ENSG00000129116,ENSG00000134686,ENSG00000108679,...,ENSG00000111716,ENSG00000166796,ENSG00000114331,ENSG00000131584,ENSG00000165410,ENSG00000172757,ENSG00000147862,ENSG00000008323,ENSG00000167083,ENSG00000149257
100001_raw-Food_weight,0.834672,-0.969366,-0.788966,-1.236243,-1.192910,-0.859681,-0.324518,0.880799,-0.152357,-1.109631,...,-0.847934,-1.246339,-1.225673,-0.270551,-0.251339,-0.348636,-0.283083,0.541040,0.412878,0.212180
100002_raw-Energy,-0.893351,-0.906648,-1.108427,-0.633443,-1.257057,-0.975325,0.100801,-0.814622,0.993700,0.709155,...,1.039015,-0.097355,0.613387,1.297104,-0.646984,-0.642464,0.254638,-0.235577,0.390156,0.875050
100003_raw-Protein,0.724878,0.367317,-1.100323,-1.304925,-0.735474,-1.178054,0.002718,-0.151997,0.730952,0.949561,...,0.668253,0.381954,0.883388,0.028914,-1.262583,-0.547308,-0.369882,0.725386,0.969339,0.582554
100004_raw-Fat,-0.273941,-1.113389,-1.176575,-0.461884,-0.947444,-0.213364,-0.169319,-1.092854,0.752200,1.207424,...,0.724525,-0.569374,0.590451,1.656459,-0.081229,-0.290437,0.928900,-0.661822,0.709848,0.600761
100005_raw-Carbohydrate,-0.925286,-0.186918,-0.488066,-0.682023,-0.746643,-0.262771,-0.261208,-0.513372,0.385347,-0.681057,...,0.692922,-0.029716,0.655211,1.326645,-0.897059,-0.569916,1.393707,-0.189605,-0.369507,0.257153


# UMAP

In [11]:
def get_umap_proj(orig_data, options):
    umap_options = {k:v for k, v in options.items() if k in DR_OPTIONS}
    display(f'UMAP options: {umap_options}')
    umap_obj = umap.UMAP(**umap_options)
    umap_obj = umap_obj.fit(orig_data)
    umap_data = umap_obj.transform(orig_data)
    return pd.DataFrame(
        data=umap_data,
        index=orig_data.index.copy(),
        columns=[f'UMAP{i+1}' for i in range(umap_data.shape[1])]
    )

In [12]:
# for n_comp, n_neigh in product(DR_OPTIONS['n_components'], DR_OPTIONS['n_neighbors']):
for n_comp in DR_OPTIONS['n_components']:
    print(f'# components: {n_comp}')
    
    options = ALL_OPTIONS.copy()
    options['n_components'] = n_comp
    
    dr_data = get_umap_proj(data, options)
    
    display(dr_data.shape)
    assert dr_data.shape == (data.shape[0], n_comp)
    
    display(dr_data.iloc[:, 0:5].describe())
    
    # save
    output_file = Path(
        RESULTS_DIR,
        generate_result_set_name(
            options,
            prefix=f'umap-{input_filepath_stem}-',
            suffix='.pkl'
        )
    ).resolve()
    display(output_file)
    
    dr_data.to_pickle(output_file)
    
    print('\n')

# components: 5


"UMAP options: {'n_components': 5, 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}"

(3749, 5)

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
count,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000
mean,9.543938,10.165720,9.601528,11.654917,10.149116
std,0.826214,0.694550,0.751955,1.428988,1.051570
min,3.332259,-0.163783,7.697813,3.638116,2.022543
25%,9.243148,9.866829,9.157655,11.503939,9.784466
50%,9.561017,10.161626,9.663405,12.009958,10.300212
75%,9.975659,10.535581,10.080146,12.449842,10.727874
max,11.436440,11.680159,12.149063,13.714686,12.997172


PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/data_transformations/umap/umap-z_score_std-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0.pkl')

...



# components: 50


"UMAP options: {'n_components': 50, 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}"

(3749, 50)

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
count,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000
mean,12.265738,10.170321,12.591548,9.513645,7.722247
std,0.972277,0.528322,0.646648,0.715110,0.562670
min,4.395242,-0.456861,8.751560,3.970074,3.659067
25%,12.336788,10.060452,12.609579,9.455735,7.656511
50%,12.501959,10.200792,12.806866,9.676305,7.847810
75%,12.641078,10.319852,12.907908,9.844992,7.977063
max,13.250747,10.810098,13.340292,10.558010,8.693202


PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/data_transformations/umap/umap-z_score_std-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_50-n_neighbors_15-random_state_0.pkl')